# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Iskateley,67.6803,53.1512,12.93,96,100,5.13,RU,1695571546
1,1,Saldanha,-33.0117,17.9442,13.73,89,99,2.39,ZA,1695571547
2,2,Karmaskaly,54.3709,56.1837,14.95,70,1,2.14,RU,1695571547
3,3,Iqaluit,63.7506,-68.5145,3.85,80,40,6.71,CA,1695571327
4,4,Tayoltita,24.0833,-105.9333,29.14,40,17,0.65,MX,1695571547


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [43]:
%%capture --no-display

map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    #tiles = "EsriUSATopo",
    #tiles = "StamenTerrain",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 1.2,
    color="City"
)

# Display the map plot
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [37]:
# Narrow down cities that fit criteria and drop any results with null values
humidity_max = 60
temperature_min = 21 # ~70F
temperature_max = 32  # ~90F 
cloudiness_max=30

# select cities where I only need a t-shirt, low humidity, and plenty of sunshine
my_cities_df = city_data_df[(city_data_df["Max Temp"] > temperature_min) & 
                            (city_data_df["Max Temp"] < temperature_max) &
                            (city_data_df["Humidity"] < humidity_max) &
                            (city_data_df["Cloudiness"] < cloudiness_max)]

my_cities_df

# Drop any rows with null values
my_cities_df = my_cities_df.dropna()

# Display sample data
my_cities_df.info()

my_cities_df.head()

<class 'pandas.core.frame.DataFrame'>
Index: 41 entries, 4 to 540
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   City_ID     41 non-null     int64  
 1   City        41 non-null     object 
 2   Lat         41 non-null     float64
 3   Lng         41 non-null     float64
 4   Max Temp    41 non-null     float64
 5   Humidity    41 non-null     int64  
 6   Cloudiness  41 non-null     int64  
 7   Wind Speed  41 non-null     float64
 8   Country     41 non-null     object 
 9   Date        41 non-null     int64  
dtypes: float64(4), int64(4), object(2)
memory usage: 3.5+ KB


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
4,4,Tayoltita,24.0833,-105.9333,29.14,40,17,0.65,MX,1695571547
66,66,Brokopondo,4.7500,-55.1667,31.57,54,2,0.34,SR,1695571552
121,121,Ribeira grande,38.5167,-28.7000,24.22,50,20,5.36,PT,1695571557
170,170,Fortuna foothills,32.6578,-114.4119,31.79,30,0,2.06,US,1695571561
174,174,Mao,39.8885,4.2658,24.31,57,20,3.09,ES,1695571563


### Step 3: Create a new DataFrame called `hotel_df`.

In [38]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = my_cities_df[["City", "Country", "Lat", "Lng", "Max Temp", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head(10)

,City,Country,Lat,Lng,Max Temp,Humidity,Hotel Name
4,Tayoltita,MX,24.0833,-105.9333,29.14,40,
66,Brokopondo,SR,4.7500,-55.1667,31.57,54,
121,Ribeira grande,PT,38.5167,-28.7000,24.22,50,
170,Fortuna foothills,US,32.6578,-114.4119,31.79,30,
174,Mao,ES,39.8885,4.2658,24.31,57,
176,Crane,US,31.3974,-102.3501,30.91,33,
180,Tabas,IR,33.5959,56.9244,27.12,9,
186,Fabens,US,31.5023,-106.1586,27.86,22,
188,Colonia,DE,50.9333,6.9500,21.57,50,
201,Chervona sloboda,UA,49.3789,32.1464,21.50,46,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [39]:
# Set parameters to search for a hotel
radius = 10000

params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key,
    "limit":1
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
print("-" * 20)
hotel_df

Starting hotel search
Tayoltita - nearest hotel: No hotel found
Brokopondo - nearest hotel: Matu eiland
Ribeira grande - nearest hotel: Hospedaria JSF
Fortuna foothills - nearest hotel: Comfort Inn
Mao - nearest hotel: Eurohotel
Crane - nearest hotel: No hotel found
Tabas - nearest hotel: هتل امیر
Fabens - nearest hotel: No hotel found
Colonia - nearest hotel: Wasserturm Hotel Cologne
Chervona sloboda - nearest hotel: Черкаси Палац
Ponta delgada - nearest hotel: Hotel Talisman
Three rivers - nearest hotel: No hotel found
Paros - nearest hotel: Hotel Acropolis
Hanceville - nearest hotel: No hotel found
Castres - nearest hotel: Kyriad
Trapani - nearest hotel: No hotel found
Celestun - nearest hotel: Hotel Flamingo Playa
Mahenge - nearest hotel: No hotel found
Villaquilambre - nearest hotel: Hotel Santiago
Hidalgo - nearest hotel: No hotel found
Pachino - nearest hotel: Hotel Celeste
Heniches'k - nearest hotel: Орхідея
Inderbor - nearest hotel: No hotel found
Ifrane - nearest hotel: la pe

,City,Country,Lat,Lng,Max Temp,Humidity,Hotel Name
4,Tayoltita,MX,24.0833,-105.9333,29.14,40,No hotel found
66,Brokopondo,SR,4.7500,-55.1667,31.57,54,Matu eiland
121,Ribeira grande,PT,38.5167,-28.7000,24.22,50,Hospedaria JSF
170,Fortuna foothills,US,32.6578,-114.4119,31.79,30,Comfort Inn
174,Mao,ES,39.8885,4.2658,24.31,57,Eurohotel
176,Crane,US,31.3974,-102.3501,30.91,33,No hotel found
180,Tabas,IR,33.5959,56.9244,27.12,9,هتل امیر
186,Fabens,US,31.5023,-106.1586,27.86,22,No hotel found
188,Colonia,DE,50.9333,6.9500,21.57,50,Wasserturm Hotel Cologne
201,Chervona sloboda,UA,49.3789,32.1464,21.50,46,Черкаси Палац


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [45]:
%%capture --no-display
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    #tiles = "EsriUSATopo",
    #tiles = "StamenTerrain",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 1.5,
    color = "City",
    hover_cols = ["Country", "Max Temp","Hotel Name"]
)

# Display the map plot
map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Max Temp,Hotel Name)